In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
class KalmanFilter(object):
    def __init__(self, A = None, C = None, Gamma = None, Sigma = None, P = None, u0 = None, V0 = None,x=None):

        if(A is None or C is None):
            raise ValueError("Set proper system dynamics.")

        self.x = x
        self.M = A.shape[1] # number of hidden states
        self.N = x.shape[0] # number of observations

        self.A = A # A is the transition probability matrix, M x M 
        self.C = C # C is the emission probability matrix, N x M
        
        self.Gamma = np.eye(self.M) if Gamma is None else Gamma # Gamma is the covariance matrix of noise term added to the hidden state transition, M x M
        self.Sigma = np.eye(self.M) if Sigma is None else Sigma # Sigma is the covariance matrix of noise term added to the emission, N x N
        
        self.P = np.zeros(self.N, self.M, self.M)
        self.P[0:-1,:,] = np.eye(self.M) if P is None else P # P is an intermediate variable during inference, N x M x M
        self.u = np.zeros(self.N, self.M) # N x M x 1
        self.V = np.zeros(self.N, self.M, self.M) # N x M x M
        self.K = np.zeros(self.N, self.M, self.N) # N x M x N
        self.c = np.zeros(self.N) # N x 1

        # for backward passing
        self.u_hat = np.zeros(self.N, self.M) # N x M x 1
        self.V_hat = np.zeros(self.N, self.M, self.M) # N x M x M
        self.J = np.zeros(self.N, self.M, self.M) # N x M x M

        self.u0 = u0 # u0 is the initial estimate of the mean of z1, M x 1
        self.V0 = V0 # V0 is the initial estimate of the variance of z1, M x M
        
        S = np.dot(np.dot(self.C, self.V0), self.C.T) + self.Sigma
        T = np.dot(self.C, self.u0)
        I = np.eye(self.M)

        self.K[0] = np.dot(np.dot(self.V0, self.C.T), np.linalg.inv(S))
        self.u[0] = self.u0 + np.dot(self.K[0], self.x[0] - np.dot(self.C, self.u0))
        self.V[0] = np.dot((I - np.dot(self.K, self.C)), self.V0)
        self.c[0] = norm.pdf(self.x[0], T, S)
    
    def forward(self,i):
        # during inference, u[n], V[n], c[n] are calculated
        self.P[i-1] = np.dot(np.dot(self.A, self.V[i-1]), self.A.T) + self.Gamma
        I = np.eye(self.M)
        S = np.dot(np.dot(self.C, self.P[i-1]), self.C.T) + self.Sigma
        T = np.dot(np.dot(self.C, self.A), self.u[i-1])
        
        self.K[i] = np.dot(np.dot(self.P[i-1], self.C.T), np.linalg.inv(S))
        self.u[i] = np.dot(self.A, self.u[i-1]) + np.dot(self.K[i], self.x[i] - T)
        self.V[i] = np.dot((I - np.dot(self.K[i], self.C)), self.P[i-1])
        self.c[i] = norm.pdf(self.x[i], T, S)

    def backward(self,i):
        self.J[i] = np.dot(np.dot(self.V[i], self.A.T), np.linalg.inv(self.P[i]))
        self.u_hat[i] = self.u[i] + np.dot(self.J[i], self.u_hat[i+1] - np.dot(self.A, self.u[-1]))
        self.V_hat[i] = self.V[i] + np.dot(np.dot(self.J[i], self.V_hat[i+1] - self.P[i]), self.J[i].T)
    def learning(self):
        self.u0 = self.u_hat[0]
        self.V0 = self.V_hat[0] + np.dot(self.u_hat[0], self.u_hat[0].T) - np.dot(self.u_hat[0], self.u_hat[0].T)

        sub_1 = []
        sub_2 = []
        sub_3 = []
        sub_4 = []
        sub_5 = []
        sub_6 = []
        sub_7 = []
        sub_8 = []
        for i in range(1,1,self.N):
            sub_1 += np.dot(self.J[i-1], self.V_hat[i]) + np.dot(self.u_hat[i],self.u_hat[i-1].T) # z[n]z[n-1]
            sub_2 += self.V_hat[i-1] + np.dot(self.u_hat[i-1], self.u_hat[i-1].T) # z[n-1]z[n-1]
            sub_3 += self.V_hat[i] + np.dot(self.u_hat[i], self.u_hat[i].T) # z[n]z[n]
            sub_4 += np.dot(self.J[i-1], self.V_hat[i]) + np.dot(self.u_hat[i-1],self.u_hat[i].T) #z[n-1]z[n]
    
        for i in range(self.N):
            sub_5 += np.dot(self.x[i], self.u_hat[i].T) # x[n]*E[z[n]]
            sub_6 += self.V_hat[i] + np.dot(self.u_hat[i], self.u_hat[i].T) # z[n]z[n]
            sub_7 += np.dot(self.x[i], self.x[i].T) # x[n]x[n]
            sub_8 += np.dot(self.u_hat[i], self.x[i].T) #E[z[n]]*x[n]

        self.A = np.dot(sub_1, np.linalg.inv(sub_2))
        self.Gamma = 1/(self.N-1) * (sub_3 - np.dot(self.A, sub_4) - np.dot(sub_1, self.A) + np.dot(np.dot(self.A, sub_2), self.A.T))
        self.C = np.dot(sub_5, np.linalg.inv(sub_6))
        self.Sigma = 1/self.N * (sub_7 - np.dot(self.C, sub_8) - np.dot(sub_5, self.C) + np.dot(np.dot(self.C, sub_6), self.C))




In [ ]:
def example():
	dt = 1.0/60
	A = np.array([[1, dt, 0], [0, 1, dt], [0, 0, 1]])
	C = np.array([1, 0, 0]).reshape(1, 3)
	Gamma = np.array([[0.05, 0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])
	Sigma = np.array([0.5]).reshape(1, 1)

	x = np.linspace(-10, 10, 100)
	measurements = - (x**2 + 2*x - 2)  + np.random.normal(0, 2, 100)

	kf = KalmanFilter(A = A, C = C, Gamma = Gamma, Sigma = Sigma)
	predictions = []

	for z in measurements:
		predictions.append(np.dot(C,  kf.predict())[0])
		kf.update(z)

	plt.plot(range(len(measurements)), measurements, label = 'Measurements')
	plt.plot(range(len(predictions)), np.array(predictions), label = 'Kalman Filter Prediction')
	plt.legend()
	plt.show()

if __name__ == '__main__':
    example()